In [1]:
import pandas as pd
import numpy as np

import torch

import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger

from pytorch_forecasting import Baseline, TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import SMAPE, PoissonLoss, QuantileLoss, MultiHorizonMetric
from pytorch_forecasting.models.temporal_fusion_transformer.tuning import optimize_hyperparameters

In [2]:
data_path = "./Refined_Data/Grouped_Data/Input_Data2.csv"

group = [
    [34, 40, 42, 41, 4, 10, 11, 12],
    [35, 6, 48, 27, 57, 8, 25, 56, 26, 55, 47, 13, 53, 18, 7, 17, 46],
    [31, 33, 9, 3, 1, 32],
    [29, 38, 43, 58, 15, 22, 39, 54, 23, 44, 45, 37, 52, 2, 14],
    [21, 19, 50, 49, 20, 51, 30, 36, 28, 59, 5, 60, 16, 24]
]

In [3]:
data = pd.read_csv(data_path, parse_dates = ["date_time"])

data['num']     =   data['num'].apply(str)
data['day_of_Week']     =   data['day_of_Week'].apply(str)
data['day_of_month']    =   data['day_of_month'].apply(str)
data['24Hour']  =   data['24Hour'].apply(str)
data['holiday'] =   data['holiday'].apply(str)
data['Weekend'] =   data['Weekend'].apply(str)
data['energy_group'] = data['energy_group'].apply(str)
data['hour_cat']=   data['hour_cat'].apply(str)
# data['discomfort_index'] = data['discomfort_index'].apply(str)

In [4]:
data_bag = [data.loc[data["energy_group"] == str(i)].copy() for i in range(len(group))]

models = []

In [5]:
for idx in range(0, 5):    

    print(group[idx])

    data = data_bag[idx]

    torch.cuda.empty_cache()

    max_prediction_length = 168
    max_encoder_length = 336
    training_cutoff = data["time_idx"].max() - max_prediction_length

    training = TimeSeriesDataSet(
        data[lambda x: x.time_idx <= training_cutoff],
        time_idx="time_idx",
        target="kWH",
        group_ids=["num"],
        min_encoder_length=max_encoder_length,
        max_encoder_length=max_encoder_length,
        min_prediction_length=max_prediction_length,
        max_prediction_length=max_prediction_length,
        static_categoricals=["num", "energy_group"],
        static_reals=["non_electric_aircondition", "sunlight"],
        time_varying_known_categoricals=["day_of_Week", "day_of_month", "24Hour", "holiday", "Weekend", "hour_cat"],
        time_varying_known_reals=["C", "m/s", "wet", "mm", "hr", "time_idx", "discomfort_real"],
        time_varying_unknown_categoricals=[],
        time_varying_unknown_reals=["kWH"],
        add_relative_time_idx=True,
        add_target_scales=True,
        add_encoder_length=True
    )

    validation = TimeSeriesDataSet.from_dataset(training, data, predict=True, stop_randomization=True)

    batch_size = 64  # set this between 32 to 128
    train_dataloader = training.to_dataloader(train=True, batch_size=batch_size, num_workers=0)
    val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size * 10, num_workers=0)

    early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=1e-4, patience=10, verbose=False, mode="min")
    lr_logger = LearningRateMonitor()  # log the learning rate
    logger = TensorBoardLogger("lightning_logs")  # logging results to a tensorboard

    trainer = pl.Trainer(
        max_epochs=500,
        gpus=1,
        weights_summary="top",
        gradient_clip_val=0.14,
        limit_train_batches=30, 
        callbacks=[lr_logger, early_stop_callback],
        logger=logger,
    )


    tft = TemporalFusionTransformer.from_dataset(
        training,
        learning_rate=0.03,
        hidden_size=64,
        lstm_layers = 1,
        attention_head_size=4,
        dropout=0.15,
        hidden_continuous_size=8,
        output_size=1,
        loss=SMAPE(),
        log_interval=0,
        reduce_on_plateau_patience=4,
    )

    # fit network
    trainer.fit(
        tft,
        train_dataloader=train_dataloader,
        val_dataloaders=val_dataloader,
    )

m=3, val_loss=0.0563, train_loss_step=0.0273, train_loss_epoch=0.0277]
Validating: 0it [00:00, ?it/s]
Epoch 42:  97%|█████████▋| 30/31 [00:16<00:00,  1.81it/s, loss=0.0276, v_num=3, val_loss=0.0566, train_loss_step=0.0295, train_loss_epoch=0.0273]
Validating: 0it [00:00, ?it/s]
Epoch 43:  97%|█████████▋| 30/31 [00:17<00:00,  1.76it/s, loss=0.0274, v_num=3, val_loss=0.0568, train_loss_step=0.0261, train_loss_epoch=0.0275]
Validating: 0it [00:00, ?it/s]
Epoch 44:  97%|█████████▋| 30/31 [00:16<00:00,  1.77it/s, loss=0.0275, v_num=3, val_loss=0.0562, train_loss_step=0.026, train_loss_epoch=0.0275] 
Validating: 0it [00:00, ?it/s]
Epoch 45:  97%|█████████▋| 30/31 [00:16<00:00,  1.80it/s, loss=0.0265, v_num=3, val_loss=0.0554, train_loss_step=0.0269, train_loss_epoch=0.0274]
Validating: 0it [00:00, ?it/s]
Epoch 46:  97%|█████████▋| 30/31 [00:16<00:00,  1.77it/s, loss=0.027, v_num=3, val_loss=0.0559, train_loss_step=0.029, train_loss_epoch=0.0268] 
Validating: 0it [00:00, ?it/s]
Epoch 47:  97%